In [1]:
import os
import urllib.request, json 
import pandas as pd
from IPython.display import HTML
from IPython import display
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
    load_petri_model,
    setup_petri_model,
    sample
)
import numpy as np
from typing import Iterable
from pyciemss.utils.interface_utils import (
    assign_interventions_to_timepoints,
    interventions_and_sampled_params_to_interval,
    convert_to_output_format
)
from pyciemss.utils import get_tspan
import matplotlib.pyplot as plt
import torch
from torch import tensor

Utils

In [3]:
def aggregate_results(samples_df, num_samples=1):
    QoIs = ["D_sol", "E_sol", "I_sol", "R_sol", "S_sol"]
    all_QoIs = {key: None for key in QoIs}
    for QoI in QoIs:
        # Aggregate results
        # TODO: After they introduce normalized models, remove normalization here.
        # /total_pop * 100
        # total_pop = 0
        # for val in QoIs:
        #     total_pop += samples_df[samples_df["sample_id"] == 0][val].to_numpy()[0]
        all_QoIs[QoI] = [samples_df[samples_df["sample_id"] == i][QoI].to_numpy() for i in range(num_samples)]
    return all_QoIs

# Add '?raw=true' at the end of the link to the file to get the rawgithubusercontent link.
def update_AMR(SEIRD_model_url, SEIRD_model_path):
    with urllib.request.urlopen(SEIRD_model_url) as url:
        json_object = json.load(url)
        with open(SEIRD_model_path, "w") as outfile:
            json.dump(json_object, outfile)

# SCENARIO 2: Multiple Vaccines and Variants

### PART 1
**(A)** TA1/TA2 provide the AMR for a COVID-19 model which includes multiple vaccines and variants of concern. \
Model Source: https://www.nature.com/articles/s41598-022-06159-x \
AMR location: https://github.com/indralab/mira/blob/hackathon/notebooks/hackathon_2023.07/scenario2_a.json \
Model Schematic:

<div>
<img src="attachment:image-3.png" width="400" align="left"/>
</div>

Load the model

In [4]:
# 1b
SEIRD_model_path = "scenario2_a_beta_scale_static.json"
SEIRD_model_url = "https://raw.githubusercontent.com/indralab/mira/hackathon/notebooks/hackathon_2023.07/scenario2_a_beta_scale_static.json"
# NOTE: This is only if you wish to update the AMR.
update_AMR(SEIRD_model_url, SEIRD_model_path)

**(B) TA3 TASK 1:** Perform model simulation over a 14-month period beginning January 1, 2020. Consider only the Wild Type variant, and no vaccination.
 - Use parameter values for wild type from supplementary material
 - 2019 population of Ontario: 14.57 million people
 - Natural death rate $\mu = 2.05e-5$
 - Initial conditions: let $I(0) = 1e-6$, $S(0) = 1 - (I(0) + A(0))$, and all other values be zero
 
Questions: 
1. When do $I(t)$ and $A(t)$ peak, and what is the value of these variables at their peaks? 
2. How do the $I(t)$ and $A(t)$ profiles compare with Fig. 3d, f (below)? 
3. Given that this question assumes only the presence of the wild type variant, do the results seem reasonable?

<div>
<img src="attachment:image-4.png" width="500" align="left"/>
</div>

In [ ]:
# NOTE: mu is set in the AMR, not sure where to find/set I(0).


### PART 2 - Extend the model to include multiple strains and (two) vaccine types
**(A) and (B)** TA1/TA2 extract and/or transform the model to include variants and vaccines. \
New schematic for transitions involving susceptible individuals:

<div>
<img src="attachment:image-2.png" width="300" align="left"/>
</div>

**(C) TA3 TASK 2:** Simulate the model over the two-year period from January 1, 2020 to December 31, 2021. 
 - Include the two vaccine types: Pfizer and AstraZeneca
 - Include three COVID-19 variants/strains: Wild Type, Alpha, and Delta (ignore Omicron for now) 
 - Initial conditions: assume the entire population of Ontario is susceptible, except for 15 infected symptomatic individuals
 - Reproduce *Figs. 3a, b, d, and e*

<div>
<img src="attachment:image-2.png" width="300" align="left"/>
</div>

### PART 3
**(D) TA3 TASK 3:** Implement the three additional scenarios described below, and used in the paper to create Fig. 7, and simulate from January 1, 2020 - December 31, 2021: 
1. Assume that a smaller percentage of vaccinations were with Pfizer, but keep the total vaccinations the same as in 2c. Given that the Pfizer vaccine offered stronger protection against all variants, compared to AstraZeneca, what is the impact?
2. Assume a lower provincial vaccination rate compared to the baseline percentage of 74% of the population vaccinated by September 1st, 2021. 
3. Assume less stringent NPIs than what actually took place. 

Qualitatively compare with simulations in Fig. 7, where: 
1. “PZ/AZ,” lower fraction of vaccines being Pfizer (from 95 to 40%), with the reduction made up with AstraZeneca, which offers reduced protection from Delta infections and a longer dosing delay
2. “Vac rate,” vaccination rate reduced by 20%
3. “NPI”, less stringent lockdown measured, with mobility increased by 25%

<div>
<img src="attachment:image.png" width="300" align="left"/>
</div>

### PART 4 - Omicron Variant
**(A)** TA2 simplifies the model structure to consider multiple vaccines, but only the Delta and Omicron variants \
**(B) TA3 TASK 4:** Using epidemiological data for Ontario (https://storage.googleapis.com/covid19-open-data/v3/location/CA_ON.csv) and summary data on Omicron-related characteristics (https://www.cdc.gov/mmwr/volumes/71/wr/mm7137a4.htm) to set relevant model parameters, simulate the simplified model from July to December 2021 and compare with the hypothetical outcomes with the fictitious Neos variant as in Fig. 9

<div>
<img src="attachment:image.png" width="600" align="left"/>
</div>

### PART 5 - Change the model setting to the United States
**(A)** TA1 updates parameter values for the US, and changes vaccine types to (1) Moderna and Pfizer mRNA and (2) Janssen/J&J viral vector vaccine. \
**(B) TA3 TASK 5:** With the updated US-specific AMR, rerun the simulation from January 1, 2020 to December 31, 2021. How do the outcomes compare with Question 2c?